In [ ]:
versioninfo()

In [ ]:
using Pkg
Pkg.activate("../..")
Pkg.status()

# Computer arithmetics

## Units of computer storage

Humans use decimal digits (why?)\
Computers use binary digits (why?)

* *Bit* = binary digit (coined by statistician [John Tukey](https://en.wikipedia.org/wiki/Bit#History)).  
* *byte* = 8 bits.
* KB = kilobyte = $10^3$ bytes; KiB = kibibyte = $2^{10}$ bytes.  
* MB = megabyte = $10^6$ bytes; MiB = mebibyte = $2^{20}$ bytes.
* GB = gigabyte = $10^9$ bytes. Typical RAM size.  
* TB = terabyte = $10^{12}$ bytes. Typical hard drive size. Size of NYSE each trading session.    
* PB = petabyte = $10^{15}$ bytes.  
* EB = exabyte = $10^{18}$ bytes. Size of all healthcare data in 2011 is ~150 EB.    
* ZB = zetabyte = $10^{21}$ bytes. 

Julia function `Base.summarysize` shows the amount of memory (in bytes) used by an object.

In [ ]:
x = rand(100, 100)
Base.summarysize(x)

`varinfo()` function prints all variables in workspace and their sizes.

In [ ]:
varinfo() # similar to Matlab whos()

## Storage of Characters

* Plain text files are stored in the form of characters: `.jl`, `.r`, `.c`, `.cpp`, `.ipynb`, `.html`, `.tex`, ...  
* ASCII (American Code for Information Interchange): 7 bits, only $2^7=128$ characters.  

In [ ]:
# integers 0, 1, ..., 127 and corresponding ascii character
[0:127 Char.(0:127)]

* Extended ASCII: 8 bits, $2^8=256$ characters.  

In [ ]:
# integers 128, 129, ..., 255 and corresponding extended ascii character
# show(STDOUT, "text/plain", [128:255 Char.(128:255)])
[128:255 Char.(128:255)]

* Unicode: UTF-8, UTF-16 and UTF-32 support many more characters including foreign characters; last 7 digits conform to ASCII. 

* [UTF-8](https://en.wikipedia.org/wiki/UTF-8) is the current dominant character encoding on internet.  

<img src="./unicode.png" width="800" align="center"/>

Source: [Google Blog](https://googleblog.blogspot.com/2012/02/unicode-over-60-percent-of-web.html)

* Julia supports the full range of UTF-8 characters. You can type many Unicode math symbols by typing the backslashed LaTeX symbol name followed by tab. 

In [ ]:
# \beta-<tab>
β = 0.0
# \beta-<tab>-\hat-<tab>
β̂ = 0.0

* For a table of unicode symbols that can be entered via tab completion of LaTeX-like abbreviations: <https://docs.julialang.org/en/v1.1/manual/unicode-input/#Unicode-Input-1>

## Integers: fixed-point number system

* Fixed-point number system is a computer model for integers $\mathbb{Z}$. 
    - **Remember** that computer memory is finite whereas the cardinality of $\mathbb{Z}$ is (countably) infinite.
    - *Any* representation of numbers in computer *has to be* an approximation.

* The number $M$ of bits and method of representing negative numbers vary from system to system. 
    - The `integer` type in R has $M=32$ (packages such as ‘bit64’support 64 bit integers). 
        + <https://www.r-bloggers.com/r-in-a-64-bit-world/>
    - C has (`unsigned`) `char`, `int`, `short`, `long` (and `long long`), whose sizes depend on the machine.
    - Matlab has `(u)int8`, `(u)int16`, `(u)int32`, `(u)int64`.  

* Julia has even more integer types: 

In [ ]:
using GraphRecipes, Plots

gr(size=(600, 400))
theme(:default)

plot(Integer, method=:tree, fontsize=4)

source: [Visualizing Graphs in Julia using Plots and PlotRecipes](http://www.breloff.com/Graphs/)

### Unsigned integers

* Model for $\mathbb{N} \cup \{0\}$.
* For unsigned integers, the range is $[0,2^M-1]$.
* Julia functions `typemin(T)` and `typemax(T)` give the lowest and highest representable number of a type `T` respectively

In [ ]:
for t in [UInt8, UInt16, UInt32, UInt64, UInt128]
    println(t, '\t', typemin(t), '\t', typemax(t))
end

### Signed integers

* Model of $\mathbb{Z}$. Can do subtraction.

* First bit ("most significant bit" or MSB) is the sign bit.  
    - `0` for nonnegative numbers
    - `1` for negative numbers  
    
* **Two's complement representation** for negative numbers. 
    - Set the sign bit to 1  
    - Negate (`0`->`1`, `1`->`0`) the remaining bits
    - Add to `1` to the result  
    - Two's complement representation of a negative integer $x$ is the same as the unsigned integer $2^M - x$.

In [ ]:
@show typeof(5)
@show bitstring(5)
@show bitstring(-5)
@show bitstring(UInt64(Int128(2)^64 - 5)) == bitstring(-5)
@show bitstring(2 * 5) # shift bits of 5 to the left
@show bitstring(2 * -5); # shift bits of -5 to left

* Two's complement representation respects modular arithmetic nicely.  
    Addition of any two signed integers are just bitwise addition, possibly modulo $2^M$
    - $M=4$ case:
    
<img src="http://users.dickinson.edu/~braught/courses/cs251f02/classes/images/twosCompWheel.png" width="400" align="center"/>    

Source: [Signed Binary Numbers, Subtraction and Overflow](http://users.dickinson.edu/~braught/courses/cs251f02/classes/notes07.html) by Grant Braught

* **Range** of representable integers by $M$-bit **signed integer** is $[-2^{M-1},2^{M-1}-1]$:

In [ ]:
typemin(Int64), typemax(Int64)

In [ ]:
for T in [Int8, Int16, Int32, Int64, Int128]
    println(T, '\t', typemin(T), '\t', typemax(T))
end

### `BigInt`

Julia `BigInt` type is arbitrary precision.

In [ ]:
@show typemax(Int128)
@show typemax(Int128) + 1 # overflow!
@show BigInt(typemax(Int128)) + 1;

### Overflow and underflow for integer arithmetic

R reports `NA` for integer overflow and underflow.  
**Julia outputs the result according to modular arithmetic.**

In [ ]:
@show typemax(Int32)
@show typemax(Int32) + Int32(1); # modular arithmetics!

In [ ]:
using RCall
# The largest integer R can hold
R"""
.Machine$integer.max 
"""

In [ ]:
R"""
M <- 32
big <- 2^(M-1) - 1
as.integer(big)
"""

In [ ]:
R"""
as.integer(big+1)
"""

## Real numbers: floating-point number system

Floating-point number system is a computer model for the real line $\mathbb{R}$.

* Most computer systems adopt the [IEEE 754 standard](https://en.wikipedia.org/wiki/IEEE_floating_point), established in 1985, for floating-point arithmetics.  
For the history, see an [interview with William Kahan](http://www.cs.berkeley.edu/~wkahan/ieee754status/754story.html).

* In the scientific notation, a real number is represented as
$$
\pm d_1.d_2d_3 \cdots d_p \times b^e, \quad 0 \le d_i < b.
$$
Humans use the _base_ $b=10$ and _digits_ $d_i=0, 1, \dotsc, 9$.\
    In computer, the base is $b=2$ and the digits $d_i$ are 0 or 1.

* **Normalized** vs **denormalized** numbers. For example, decimal number 18 is
$$ +1.0010 \times 2^4 \quad (\text{normalized})$$
or, equivalently,
$$ +0.1001 \times 2^5 \quad (\text{denormalized}).$$

* In the floating-number system, computer stores 
    - sign bit  
    - the _fraction_ (or _mantissa_, or _significand_) of the **normalized** representation
    - the actual exponent _plus_ a bias

* Julia provides `Float16` (half precision, implemented in software using `Float32`), `Float32` (single precision), `Float64` (double precision), and `BigFloat` (arbitrary precision).

In [ ]:
using GraphRecipes, Plots

#pyplot(size=(800, 600))
gr(size=(600, 400))
theme(:default)

plot(AbstractFloat, method=:tree, fontsize=4)

### Half precision (Float16)

<img src="./half-precision-numbers.png" width="300" align="center"/>

Source: <https://en.wikipedia.org/wiki/Half-precision_floating-point_format>
    
- In Julia, `Float16` is the type for half precision numbers.

- MSB is the sign bit.  

- 10 significand bits (**fraction**=**mantissa**), hence $p=11$ (why?)

- 5 exponent bits: $e_{\max}=15$, $e_{\min}=-14$, **bias**=15 = $01111_2$ for encoding:
    + $e_{\min} = \mathbf{00001_2} - 01111_2 = -14_{10}$
    + $e_{\max} = \mathbf{11110_2} - 01111_2 = 15_{10}$

- $e_{\text{min}}-1$ and $e_{\text{max}}+1$ are reserved for special numbers.  

- range of **magnitude**: $10^{\pm 4}$ in decimal because $\log_{10} (2^{15}) \approx 4$.  

- **Precision**: $\log_{10}2^{11} \approx 3.311$ decimal digits. 

$$
(value) = (-1)^{b_{15}}\times 2^{(\sum_{j=1}^5 b_{15-j}2^{5-j}) - 15} \times \left( 1 + \sum_{i=1}^{10}\frac{b_{10-i}}{2^i}\right)
$$

In [ ]:
println("Half precision:")
@show bitstring(Float16(5)) # 5 in half precision
@show bitstring(Float16(-5)); # -5 in half precision

### Single precision (Float32)

<img src="./single-precision-numbers.png" width="600" align="center"/>

Source: <https://en.wikipedia.org/wiki/Single-precision_floating-point_format>

- In Julia, `Float32` is the type for single precision numbers.  

- MSB is the sign bit.  

- 23 significand bits ($p=24$).  

- 8 exponent bits: $e_{\max}=127$, $e_{\min}=-126$, **bias**=127.  

- $e_{\text{min}}-1$ and $e_{\text{max}}+1$ are reserved for special numbers.  

- range of **magnitude**: $10^{\pm 38}$ in decimal because $\log_{10} (2^{127}) \approx 38$.

- **precision**: $\log_{10}(2^{24}) \approx 7.225$ decimal digits.  

In [ ]:
println("Single precision:")
@show bitstring(Float32(5)) # 5 in single precision
@show bitstring(Float32(-5)); # -5 in single precision

### Double precision (Float64)

<img src="./double-precision-numbers.png" width="600" align="center"/>

Source: <https://en.wikipedia.org/wiki/Double-precision_floating-point_format>

- Double precision (64 bits = 8 bytes) numbers are the dominant data type in scientific computing.
      
- In Julia, `Float64` is the type for double precision numbers.    

- MSB is the sign bit.  

- 52 significand bits ($p=53$).

- 11 exponent bits: $e_{\max}=1023$, $e_{\min}=-1022$, **bias**=1023.  

- $e_{\text{min}}-1$ and $e_{\text{max}}+1$ are reserved for special numbers.  

- range of **magnitude**: $10^{\pm 308}$ in decimal because $\log_{10} (2^{1023}) \approx 308$.  

- **precision** to the $\log_{10}(2^{53}) \approx 15.95$ decimal point.

In [ ]:
println("Double precision:")
@show bitstring(Float64(5)) # 5 in double precision
@show bitstring(Float64(-5)); # -5 in double precision

### Special floating-point numbers

- Exponent $e_{\max}+1$ plus a zero mantissa means $\pm \infty$.

In [ ]:
@show bitstring(Inf) # Inf in double precision
@show bitstring(-Inf); # -Inf in double precision

- Exponent $e_{\max}+1$ plus a nonzero mantissa means `NaN`. `NaN` could be produced from `0 / 0`, `0 * Inf`, ...  

- In general `NaN ≠ NaN` bitwise. Test whether a number is `NaN` by `isnan` function.  

In [ ]:
@show bitstring(0 / 0) # NaN
@show bitstring(0 * Inf); # NaN

- Exponent $e_{\min}-1$ with a zero mantissa represents the real number 0 ("exact zero").  
    + Why do we need an exact zero?

In [ ]:
@show bitstring(0.0); # 0 in double precision 

- Exponent $e_{\min}-1$ with a nonzero mantissa are for numbers less than $b^{e_{\min}}$.  
    Numbers are _denormalized_ in the range $(0,b^{e_{\min}})$ -- **gradual underflow**. 
- For example, in half-precision, $e_{\min}=-14$ but $2^{-24}$ is represented by $0.0000000001_2 \times 2^{-14}$.

In [ ]:
@show Float16(2^(-14))  # emin=-14
@show bitstring(Float16(2^(-14)));
@show Float16(2^(-24))  # emin=-14
@show bitstring(Float16(2^(-24))); # denormalized

In [ ]:
@show nextfloat(Float16(0.0)) # next representable number
@show bitstring(nextfloat(Float16(0.0))); # denormalized

### Rounding

* Rounding is necessary whenever a number has more than $p$ significand bits. Most computer systems use the default IEEE 754 _round to nearest_ mode (also called _ties to even_ mode). Julia offers several [rounding modes](https://docs.julialang.org/en/v1/base/math/#Base.Rounding.RoundingMode), the default being [`RoundNearest`](https://docs.julialang.org/en/v1/base/math/#Base.Rounding.RoundNearest). 

* "*Round to nearest, ties to even*" rule: rounds to the nearest value; if the number falls midway, it is rounded to the nearest value with an even least significant digit (default for IEEE 754 binary floating point numbers)

* For example, the number 1/10 cannot be represented accurately as a (binary) floating point number:
$$ 0.1 = 1.10011001\dotsc_2 \times 2^{-4} $$

In [ ]:
@show bitstring(0.1);  # double precision Float64
@show bitstring(0.1f0); # single precision Float32, 1001 gets rounded to 101(0)
@show bitstring(Float16(0.1)); # half precision Float16, 1001 gets rounded to 101(0)

### Errors

Rounding (more fundamentally, finite precision) incurs errors in floating porint computation. If a real number $x$ is represented by a floating point number $[x]$, then

* Absolute error
$$
    | [x] - x |
$$

* Relative error
$$
    \frac{| [x] - x |}{|x|}
$$
(if $x \neq 0$).

Of course, we want to ensure that the error after a computation is small.

### Machine epsilons

- Floating-point numbers do not occur uniformly over the real number line
    <img src="http://www.volkerschatz.com/science/pics/fltscale-wh.png" width="700" align="center"/>
    
    Source: [What you never wanted to know about floating point but will be forced to find out](http://www.volkerschatz.com/science/float.html)
    
- Same number of representible numbers in $(2^i, 2^{i+1}]$ as in $(2^{i+1}, 2^{i+2}]$. Within an interval, they are uniformly distributed.
    
- **Machine epsilons** are the spacings of numbers around 1: 
    + $\epsilon_{\max}$ = (smallest positive floating point number that added to 1 will give a result different from 1) = $\frac{1}{2^p} + \frac{1}{2^{2p-1}}$
    + $\epsilon_{\min}$ = (smallest positive floating point number that subtracted from 1 will give a result different from 1) = $\frac{1}{2^{p+1}} + \frac{1}{2^{2p}}$.
    
    <img src="./machine_epsilons.png" width="500" align="center"/>

Source: *Computational Statistics*, James Gentle, Springer, New York, 2009.
    
- Any real number in the interval $\left[1 - \frac{1}{2^{p+1}}, 1 + \frac{1}{2^p}\right]$ is represented by a floating point number $1 = 1.00\dotsc 0_2 \times 2^0$ (assuming the "ties to even" rule: consider $p=2$ case).

- Adding $\frac{1}{2^p}$ to 1 won't reach the next representable floating point number  $1.00\dotsc 01_2 \times 2^0 = 1 + \frac{1}{2^{p-1}}$. Hence $\epsilon_{\max} > \frac{1}{2^p} = 1.00\dotsc 0_2 \times 2^{-p}$.

- Adding the floating point number next to $\frac{1}{2^p} = 1.00\dotsc 0_2 \times 2^{-p}$ to 1 WILL result in $1.00\dotsc 01_2 \times 2^0 = 1 + \frac{1}{2^{p-1}}$, hence $\epsilon_{\max} = 1.00\dotsc 01_2 \times 2^{-p} = \frac{1}{2^p} + \frac{1}{2^{p+p-1}}$.

- Subtracting $\frac{1}{2^{p+1}}$ from 1 results in $1-\frac{1}{2^{p+1}} = \frac{1}{2} + \frac{1}{2^2} + \dotsb + \frac{1}{2^p} + \frac{1}{2^{p+1}}$, which is represented by the floating point number $1.00\dotsc 0_2 \times 2^{0} = 1$ by the "ties to even" rule. Hence $\epsilon_{\min} > \frac{1}{2^{p+1}}$.

- The smallest positive floating point number larger than $\frac{1}{2^{p+1}}$ is $\frac{1}{2^{p+1}} + \frac{1}{2^{2p}}=1.00\dotsc 1_2 \times 2^{-p-1}$. Thus $\epsilon_{\min} = \frac{1}{2^{p+1}} + \frac{1}{2^{2p}}$.

### Machine precision

* Machine epsilon is often called the machine precision.

* If a positive real number $x \in \mathbb{R}$ is represented by $[x]$ in the floating point arithmetic, then 
$$
    [x] = \left( 1 + \sum_{i=1}^{p-1}\frac{b_{i+1}}{2^i}\right) \times 2^e.
$$
Thus $x - \frac{2^e}{2^p} < [x] \le x + \frac{2^e}{2^p}$, 
and
$$
    \begin{split}
    \frac{| x - [x] |}{|x|} &\le \frac{2^e}{2^p|x|} \le \frac{2^e}{2^p}\frac{1}{[x]-2^e/2^p} \\
                            &\le \frac{2^e}{2^p}\frac{1}{2^e(1-1/2^p)}  \quad (\because [x] \ge 2^e) \\
                            &\le \frac{2^e}{2^p}\frac{1}{2^e}(1 + \frac{1}{2^{p-1}}) \\
                            &= \frac{1}{2^p} + \frac{1}{2^{2p-1}} = \epsilon_{\max}.
    \end{split}
$$
That is, the **relative error** of the floating point representation $[x]$ of real number $x$ is bounded by $\epsilon_{\max}$.

In [ ]:
@show 2^(-53) + 2^(-105);   # epsilon_max for Float64
@show 1.0 + 2^(-53);
@show 1.0 + (2^(-53) + 2^(-105));
@show 1.0 + 2^(-53) + 2^(-105);  # why is the result?  See "Catastrophic cancellation"

@show Float32(2^(-24) + 2^(-47)); # epsilon_max for Float32
@show 1.0f0 + Float32(2^(-24));
@show 1.0f0 + Float32(2^(-24) + 2^(-47));

In [ ]:
@show 2^(-54) + 2^(-106);  # epsilon_min for Float64
@show 1 - (2^(-54) + 2^(-106))
@show bitstring(1.0)
@show bitstring(1 - (2^(-54) + 2^(-106)))

In Julia, `eps(x)` gives the distance between consecutive representable floating point values at `x`, i.e.,
```Julia
eps(x) == max(x-prevfloat(x), nextfloat(x)-x)
```
which is roughly twice the $\epsilon_{\max}$.

In [ ]:
@show eps(Float32)  # machine epsilon for a floating point type
@show eps(Float64)  # same as eps()
# eps(x) is the spacing after x
@show eps(100.0)
@show eps(0.0)
# nextfloat(x) and prevfloat(x) give the neighbors of x
@show x = 1.25f0
@show prevfloat(x), x, nextfloat(x)
@show bitstring(prevfloat(x)), bitstring(x), bitstring(nextfloat(x));

* In R, the variable `.Machine` contains numerical characteristics of the machine. `double.neg.eps` is our $\epsilon_{\max}$.

In [ ]:
R"""
.Machine
"""

### Overflow and underflow of floating-point number

* For double precision, the range is $\pm 10^{\pm 308}$. In most situations, underflow (magnitude of result is less than $10^{-308}$) is preferred over overflow (magnitude of result is larger than $10^{+308}$). Overflow produces $\pm \inf$. Underflow yields zeros or subnormal numbers. 

* E.g., the logit link function is
$$p = \frac{\exp (x^T \beta)}{1 + \exp (x^T \beta)} = \frac{1}{1+\exp(- x^T \beta)}.$$
The former expression can easily lead to `Inf / Inf = NaN`, while the latter expression leads to gradual underflow.

* `floatmin` and `floatmax` functions gives largest and smallest _non-submormal_ number represented by the given floating point type.

In [ ]:
for T in [Float16, Float32, Float64]
    println(T, '\t', floatmin(T), '\t', floatmax(T), '\t', typemin(T), 
        '\t', typemax(T), '\t', eps(T))
end

* `BigFloat` in Julia offers arbitrary precision.

In [ ]:
@show precision(BigFloat)
@show floatmin(BigFloat)
@show floatmax(BigFloat);

In [ ]:
@show BigFloat(π); # default precision for BigFloat is 256 bits
# set precision to 1024 bits
setprecision(BigFloat, 1024) do 
    @show BigFloat(π)
end;

## Catastrophic cancellation

* **Scenario 1**: Addition or subtraction of two numbers of widely different magnitudes: $a+b$ or $a-b$ where $a \gg b$ or $a \ll b$. We lose the precision in the number of smaller magnitude. Consider 
$$\begin{eqnarray*}
    a &=& x.xxx ... \times 2^{30} \\  
    b &=& y.yyy... \times 2^{-30}
\end{eqnarray*}$$
What happens when computer calculates $a+b$? We get $a+b=a$!

In [ ]:
@show a = 2.0^30
@show b = 2.0^-30
@show a + b == a
@show bitstring(a)
@show bitstring(a + b);

Analysis: suppose we want to compute $x + y$, $x, y > 0$. Let the relative error of $x$ and $y$ be
$$
\delta_x = \frac{[x] - x}{x},
\quad
\delta_y = \frac{[y] - y}{y}
.
$$
What the computer actually calculates is $[x] + [y]$, which in turn is represented by $[ [x] + [y] ]$. The relative error of this representation is
$$
\delta_{\text{sum}} = \frac{[[x]+[y]] - ([x]+[y])}{[x]+[y]}
.
$$
Recall that $|\delta_x|, |\delta_y|, |\delta_{\text{sum}}| \le \epsilon_{\max}$.

We want to find a bound of the relative error of $[[x]+[y]]$ with respect to $x+y$.
Since $|[x]+[y]| = |x(1+\delta_x) + y(1+\delta_y)| \le |x+y|(1+\epsilon_{\max})$, we have
$$
\begin{split}
| [[x]+[y]]-(x+y) | &= | [[x]+[y]] - [x] - [y] + [x] - x + [y] - y | \\
                   &\le | [[x]+[y]] - [x] - [y] | +  | [x] - x | + | [y] - y | \\
                   &\le |\delta_{\text{sum}}([x]+[y])| + |\delta_x x| + |\delta_y y| \\
                   &\le \epsilon_{\max}(x+y)(1+\epsilon_{\max}) + \epsilon_{\max}x + \epsilon_{\max}y \\
                   &\approx 2\epsilon_{\max}(x+y)
\end{split}
$$
because $\epsilon_{\max}^2 \approx 0$. Thus
$$
\frac{| [[x]+[y]]-(x+y) |}{|x+y|} \le 2\epsilon_{\max}
$$
approximately.

* **Scenario 2**: Subtraction of two nearly equal numbers eliminates significant digits.  $a-b$ where $a \approx b$. Consider 
$$\begin{eqnarray*}
    a &=& x.xxxxxxxxxx1ssss  \\
    b &=& x.xxxxxxxxxx0tttt
\end{eqnarray*}$$
The result is $1.vvvvu...u$ where $u$ are unassigned digits.

In [ ]:
a = 1.2345678f0 # rounding
@show bitstring(a) # rounding
b = 1.2345677f0
@show bitstring(b)
@show a - b # correct result should be 1f-7
@show bitstring(a - b)   # must be 1.0000...0 x 2^(-23)
@show Float32(1/2^23);

Analysis: Let
$$
[x] = 1 + \sum_{i=1}^{p-2}\frac{d_{i+1}}{2^i} + \frac{1}{2^{p-1}},
\quad
[y] = 1 + \sum_{i=1}^{p-2}\frac{d_{i+1}}{2^i} + \frac{0}{2^{p-1}}
.
$$

* $[x]-[y] = \frac{1}{2^{p-1}} = [[x]-[y]]$.

* The true difference $x-y$ may lie anywhere in $(0, \frac{1}{2^{p-2}}+\frac{1}{2^{2p}}]$.

* Relative error 
$$
\frac{|x-y -[[x]-[y]]|}{|x-y|}
$$
is unbounded -- no guarantee of any significant digit!

* Implications for numerical computation
    - Rule 1: add small numbers together before adding larger ones  
    - Rule 2: add numbers of like magnitude together (paring). When all numbers are of same sign and similar magnitude, add in pairs so each stage the summands are of similar magnitude  
    - Rule 3: avoid substraction of two numbers that are nearly equal

### Algebraic laws

Floating-point numbers may violate many algebraic laws we are familiar with, such associative and distributive laws. See the example in the Machine Epsilon section and HW1.

## Coditioning

Condiser solving a linear system $Ax=b$:

$$
\begin{bmatrix} 1.000 & 0.500 \\ 0.667 & 0.333 \end{bmatrix}
\begin{bmatrix} x_1 \\ x_2 \end{bmatrix}
=
\begin{bmatrix} 1.500 \\ 1.000 \end{bmatrix}
,
$$

whose solution is $(x_1, x_2) = (1.000, 1.000)$.

In [ ]:
A=[1.0 0.5; 0.667 0.333]; b = [1.5, 1.0]; A\b

If we *perturb* $b$ by 0.001, i.e., solve

$$
\begin{bmatrix} 1.000 & 0.500 \\ 0.667 & 0.333 \end{bmatrix}
\begin{bmatrix} x_1 \\ x_2 \end{bmatrix}
=
\begin{bmatrix} 1.500 \\ 0.999 \end{bmatrix}
,
$$

then the solution changes to $(x_1, x_2) = (0.000, 3.000)$.

In [ ]:
b1 = [1.5, 0.999]; A\b1

If we instead perturb $A$ by 0.001, i.e., solve

$$
\begin{bmatrix} 1.000 & 0.500 \\ 0.667 & 0.334 \end{bmatrix}
\begin{bmatrix} x_1 \\ x_2 \end{bmatrix}
=
\begin{bmatrix} 1.500 \\ 1.000 \end{bmatrix}
,
$$

then this time the solution changes to $(x_1, x_2) = (2.000, -1.000)$.

In [ ]:
A1=[1.0 0.5; 0.667 0.334]; A1\b

In other words, an input perturbation of order $10^{-3}$ yield an output perturbation of order $10^0$. Thats 3 orders of magnutides of relative change!

Floating point representation $[x]$ of a real number $x$ in a digital computer may introduce such input perturbation easily. The perturbation of output of a problem with respect to the input is called *conditioning*.

* Abstractly, a *problem* can be viewed as function $f: X \to Y$ where $X$ is a normed vector space of data and $Y$ is a normed vector space of solutions.
    - The problem of solving $Ax=b$ for fixed $b$ is $f: A \mapsto A^{-1}b$ with $X=\{M\in\mathbb{R}^{n\times n}: M \text{ is invertible} \}$ and $Y = \mathbb{R}^n$.
    - The combination of a problem $f$ with a given data $x$ is called a *problem instance*, or simply problem unless no confusion occurs.
    
* A *well-conditioned* problem (instance) is one such that all small perturbations of $x$ lead to only small changes in $f(x)$.

* An *ill-conditioned* problem is one such that some small perturbation of $x$ leads to a large change in $f(x)$.

* The (relative) *condition number* $\kappa=\kappa(x)$ of a problem is defined by
$$
    \kappa = \lim_{\delta\to 0}\sup_{\|\delta x\|\le \delta}\frac{\|\delta f\|/\|f(x)\|}{\|\delta x\|/\|x\|}
    = \sup_{\delta x} \frac{\|\delta f\|/\|f(x)\|}{\|\delta x\|/\|x\|}
$$

* For the problem of solving $Ax=b$ for fixed $b$,  $f: A \mapsto A^{-1}b$, it can be shown that the condition number of $f$ is
$$
    \kappa = \|A\|\|A^{-1}\| =: \kappa(A)
    ,
$$
where $\|A\|$ is the matrix norm induced by vector norm $\|\cdot\|$, i.e.,
$$
    \|A\| = \sup_{x\neq 0} \frac{\|Ax\|}{\|x\|}.
$$
If 2-norm is used, then 
$$
\kappa(A) = \sigma_{\max}(A)/\sigma_{\min}(A),
$$
the ratio of the maximum and minimum singular values of $A$. To see this, let $x=A^{-1}b$. Then perturbation $A \to A+\delta A$ yields $x \to x + \delta x$. If both are small,
$$
(\delta A)x + A(\delta x) = 0
$$
or $\delta x = -A^{-1}(\delta A) x$. Then
$$
\frac{\|\delta x\|/\|x\|}{\|\delta A\|/\|A\|} \le \frac{\|A^{-1}\|\|\delta A\|\|A\|}{\|\delta A\|}
= \|A^{-1}\|\|A\|.
$$
The inequality holds with equality if $\delta A$ satisfies 
$$
\|A^{-1}(\delta A)x \| = \|A^{-1}\|\|\delta A\|\|x\|
.
$$
It can be shown that for any invertible $A$ and $b$, such perturbation $\delta A$ exists (HW2).

In the above problem, the condition number is matrix $A$ (w.r.t. Euclidean norm) is

In [ ]:
using LinearAlgebra
LinearAlgebra.cond(A)

## Further readings

* Section II.2, [Computational Statistics](https://link.springer.com/book/10.1007%2F978-0-387-98144-4) by James Gentle (2009).

* Sections 1.5 and 2.2, [Applied Numerical Linear Algebra](https://doi.org/10.1137/1.9781611971446) by James W. Demmel (1997).

* [What every computer scientist should know about floating-point arithmetic](https://www.itu.dk/~sestoft/bachelor/IEEE754_article.pdf) by David Goldberg (1991).

## Acknowledgment

Many parts of this lecture note is based on [Dr. Hua Zhou](http://hua-zhou.github.io)'s 2019 Spring Statistical Computing course notes available at <http://hua-zhou.github.io/teaching/biostatm280-2019spring/index.html>.